In [33]:
import itertools
#PyTorch
import torch

In [47]:
cifar10_dir = "/cluster/tufts/hugheslab/eharve06/CIFAR-10"
cifar101_v4_dir = "/cluster/tufts/hugheslab/eharve06/CIFAR-10.1"
ns = [100, 1_000, 10_000, 50_000]
random_states = [1001, 2001, 3001]
repo_dir = "/cluster/tufts/hugheslab/eharve06/random-Fourier-features"

for n, random_state in itertools.product(ns, random_states):
    encoded_path = f"{repo_dir}/datasets/CIFAR-10/n={n}_random_state={random_state}.pth"
    print(
        f"    \'python ../src/encode_cifar10.py "
        f"--batch_size=128 "
        f"--cifar10_dir=\"{cifar10_dir}\" "
        f"--cifar101_v4_dir=\"{cifar101_v4_dir}\" "
        f"--encoded_path=\"{encoded_path}\" "
        f"--n={n} "
        f"--num_workers=0 "
        f"--random_state={random_state}\'"
    )


    'python ../src/encode_cifar10.py --batch_size=128 --cifar10_dir="/cluster/tufts/hugheslab/eharve06/CIFAR-10" --cifar101_v4_dir="/cluster/tufts/hugheslab/eharve06/CIFAR-10.1" --encoded_path="/cluster/tufts/hugheslab/eharve06/random-Fourier-features/datasets/CIFAR-10/n=100_random_state=1001.pth" --n=100 --num_workers=0 --random_state=1001'
    'python ../src/encode_cifar10.py --batch_size=128 --cifar10_dir="/cluster/tufts/hugheslab/eharve06/CIFAR-10" --cifar101_v4_dir="/cluster/tufts/hugheslab/eharve06/CIFAR-10.1" --encoded_path="/cluster/tufts/hugheslab/eharve06/random-Fourier-features/datasets/CIFAR-10/n=100_random_state=2001.pth" --n=100 --num_workers=0 --random_state=2001'
    'python ../src/encode_cifar10.py --batch_size=128 --cifar10_dir="/cluster/tufts/hugheslab/eharve06/CIFAR-10" --cifar101_v4_dir="/cluster/tufts/hugheslab/eharve06/CIFAR-10.1" --encoded_path="/cluster/tufts/hugheslab/eharve06/random-Fourier-features/datasets/CIFAR-10/n=100_random_state=3001.pth" --n=100 --num

In [51]:
n, random_state = 1000, 1001
dataset = torch.load(f"{repo_dir}/datasets/CIFAR-10/n={n}_random_state={random_state}.pth", map_location=torch.device("cpu"), weights_only=False)
X_train, y_train = dataset["X_train"], dataset["y_train"]
X_val, y_val = dataset["X_val"], dataset["y_val"]
X_test, y_test = dataset["X_test"], dataset["y_test"]
X_ood, y_ood = dataset["X_ood"], dataset["y_ood"]


In [53]:
print(X_ood.shape)

torch.Size([2021, 2048])
